In [1]:
import numpy as np

from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
def gbdt_lr_train_test(libsvmFileName):
    #GBDT
    start = time.clock()
    train_df, test_df = train_test_split(example, train_size = 0.8,random_state=42)
    X_train = train_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_train = train_df['Grant.Status']
    X_test = test_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_test = test_df['Grant.Status']
    param_test1 = {'n_estimators':range(20,81,10)}
    gbclf = GradientBoostingClassifier(n_estimators=30, max_depth=4, verbose=0, random_state=10)
    
    
    
    tuned_parameter = [{'n_estimators':[100], 'max_depth':[ 6, 7], 'max_features':[0.5]}]
    gs_clf = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10)
    gs_clf.fit(X_train, y_train)
    print('best parameters set found: ')
    print(gs_clf.best_params_)
    y_pred_gbdt = gs_clf.predict_proba(X_test)[:, 1]
    gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)
    print('gbdt auc: %.5f' % gbdt_auc)
    gbclf=GradientBoostingClassifier(**gs_clf.best_params_, random_state=10)
    gbclf.fit(X_train,y_train)
    X_train_leaves = gbclf.apply(X_train)[:,:,0]
    (train_rows, cols) = X_train_leaves.shape
    gbclf.fit(X_test,y_test)
    X_test_leaves = gbclf.apply(X_test)[:,:,0]
    gbdtenc = OneHotEncoder()
    X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))
    #GBDT+LR
    lr = LogisticRegression()
    lr.fit(X_trans[:train_rows, :], y_train)
    y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_trans[:train_rows, :], X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_trans[train_rows:, :], X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('GBDT time taken: %.2f'% f_time)
    '''#+NB
    gnb= GaussianNB()
    gnb.fit(X_trans[:train_rows, :], y_train)
    Y_pred_nb=gbn.predict(X_trans[train_rows:, :])[:, 1]
    gnb_auc = roc_auc_score(y_test,Y_pred_nb)
    print('NB auc: ', gnb_auc)'''
    start = time.clock()
    #svc
    svc=SVC(probability=True)

    svc.fit(X_trans[:train_rows, :], y_train)
    Y_pred_svc=svc.predict_proba(X_trans[train_rows:, :])[:, 1]
    svc_auc=roc_auc_score(y_test,Y_pred_svc)
    print('SVC auc: %.5f' % svc_auc)
    svc.fit(X_train_ext, y_train)
    y_pred_svc2=svc.predict_proba(X_test_ext)[:, 1]
    svc_auc2=roc_auc_score(y_test,y_pred_svc2)
    print('SVC auc2: %.5f' % svc_auc2)
    #KNN
    knn=KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_trans[:train_rows, :], y_train)
    Y_pred_knn=knn.predict_proba(X_trans[train_rows:, :])[:, 1]
    knn_auc=roc_auc_score(y_test,Y_pred_knn)
    print('KNN auc : %.5f' % knn_auc)
    knn.fit(X_train_ext, y_train)
    y_pred_knn2=knn.predict_proba(X_test_ext)[:, 1]
    knn_auc2=roc_auc_score(y_test,y_pred_knn2)
    print('KNN auc2: %.5f' % knn_auc2)
    '''#perceptron
    perceptron = Perceptron()
    perceptron.fit(X_trans[:train_rows, :], y_train)
    Y_pred_prec=perceptron.predict(X_trans[train_rows:, :])[:, 1]
    perc_auc=roc_auc_score(y_test,Y_pred_prec)
    print('Perceptron auc : ', perc_auc)'''
    '''#linear svc
    lin = LinearSVC()
    lin.fit(X_trans[:train_rows, :], y_train)
    Y_pred_lin=lin.predict(X_trans[train_rows:, :])[:, 1]
    lin_auc=roc_auc_score(y_test,Y_pred_lin)
    print('Linear SVC auc : ', lin_auc)'''
    #SGD
    sgd = SGDClassifier(loss='log')
    sgd.fit(X_trans[:train_rows, :], y_train)
    Y_pred_sgd=sgd.predict_proba(X_trans[train_rows:, :])[:, 1]
    sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
    print('SGD auc : %.5f' % sgd_auc)
    sgd.fit(X_trans[:train_rows, :], y_train)
    Y_pred_sgd2=sgd.predict_proba(X_trans[train_rows:, :])[:, 1]
    sgd_auc2=roc_auc_score(y_test,Y_pred_sgd2)
    print('SGD auc2 : %.5f' % sgd_auc2)
    f_time =time.clock()-start
    print('other classfier time taken: %.2f'% f_time)    
    
    #XGB
    start = time.clock()
    xgb=XGBClassifier()
    xgb.fit(X_train,y_train)
    Y_pred_xgb=xgb.predict(X_test)
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    
    print('XGB auc : %.5f' % xgb_auc)
    f_time =time.clock()-start
    print('XGB time taken: %.2f'% f_time)
    #lightGBM
    '''start = time.clock()
    
    print('lightGBM time taken: ', time.clock()-start)'''
    
    

In [ ]:
def param_test1(file):
    param_test1 = {'n_estimators':range(20,81,10)}
    gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
    param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
    gsearch1.fit(train[predictors],train[target])
    gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
def param_test2(file):
    param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
    gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, max_features='sqrt', subsample=0.8, random_state=10), 
    param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
    gsearch2.fit(train[predictors],train[target])
    gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
def param_test3(file):
    param_test3 = {'min_samples_split':range(1000,2100,200), 'min_samples_leaf':range(30,71,10)}
    gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9,max_features='sqrt', subsample=0.8, random_state=10), 
    param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
    gsearch3.fit(train[predictors],train[target])
    gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_


In [ ]:
def param_test4(file):
    param_test4 = {'max_features':range(7,20,2)}
    gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.8, random_state=10),
    param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
    gsearch4.fit(train[predictors],train[target])
    gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [4]:
example = pd.read_csv('example.csv')